# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [22]:
## The following libraries might be useful
# !pip install -q langchain-openai
# !pip install -U -q langchain-community
# !pip install -U -q langchain-chroma
# !pip install -U -q datasets
# !pip install -U -q ragas
# !pip install -U -q rouge_score

In [23]:
# Import essential libraries
!pip install langchain llama-index openai rouge-score ragas evaluate nltk scikit-learn tiktoken
!pip install git+https://github.com/explodinggradients/ragas.git


  Cloning https://github.com/explodinggradients/ragas.git to /tmp/pip-req-build-fb3z1yfz
  Running command git clone --filter=blob:none --quiet https://github.com/explodinggradients/ragas.git /tmp/pip-req-build-fb3z1yfz
  Resolved https://github.com/explodinggradients/ragas.git to commit daa8ca20f1bb2592a3eeadd8364bb55ca0934a99
ERROR: git+https://github.com/explodinggradients/ragas.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [24]:
import zipfile
import os

# Step 1: Set path for the uploaded ZIP file and target extraction directory
zip_path = "/content/rag_legal.zip"
extract_dir = "/content/rag_legal"  # this will be your root dataset folder

# Step 2: Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Dataset unzipped at:", extract_dir)


✅ Dataset unzipped at: /content/rag_legal


In [25]:
import os
os.listdir("/content/rag_legal")


['rag_legal']

In [26]:
# Load the files as documents
import glob
import os

def load_all_txt_documents(folder_path):
    all_docs = []

    # Recursively search for all .txt files
    for file_path in glob.glob(os.path.join(folder_path, '**', '*.txt'), recursive=True):
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read()
                all_docs.append({
                    "text": text,
                    "file_path": file_path,
                    "category": os.path.basename(os.path.dirname(file_path))
                })
        except Exception as e:
            print(f"❌ Error reading {file_path}: {e}")

    return all_docs

corpus_path = "/content/rag_legal"
documents = load_all_txt_documents(corpus_path)

print(f"✅ Loaded {len(documents)} documents")
print("📄 Sample document keys:", documents[0].keys() if documents else "No documents found")



✅ Loaded 698 documents
📄 Sample document keys: dict_keys(['text', 'file_path', 'category'])


#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [27]:
# Clean and preprocess the data

# STEP 1: Imports and stopwords setup
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from langchain.schema import Document

stop_words = set(stopwords.words('english'))

# STEP 2: Text cleaning function
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\S+@\S+', '', text)  # Remove emails
    text = re.sub(r'(\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4})', '', text)  # Remove phone numbers
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove special characters
    text = text.lower()
    return ' '.join([word for word in text.split() if word not in stop_words])

# STEP 3: Create LangChain documents from preprocessed raw data
# documents already exists with keys: 'text', 'file_path', 'category'
docs = [
    Document(
        page_content=preprocess_text(doc['text']),
        metadata={"source": doc["file_path"], "category": doc["category"]}
    )
    for doc in documents
]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [28]:
# Calculate the average, maximum and minimum document length.
from collections import Counter
import numpy as np

lengths = [len(doc.page_content.split()) for doc in docs]
print(f"Average Length: {np.mean(lengths)}\nMax Length: {np.max(lengths)}\nMin Length: {np.min(lengths)}")

Average Length: 8975.432664756447
Max Length: 84946
Min Length: 142


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [29]:
# Find frequency of occurence of words
all_words = ' '.join(doc.page_content for doc in docs).split()
counter = Counter(all_words)

print("20 Most Common Words:", counter.most_common(20))
print("20 Least Common Words:", counter.most_common()[-20:])

20 Most Common Words: [('company', 148170), ('shall', 107995), ('agreement', 104559), ('section', 75344), ('parent', 58009), ('party', 49657), ('date', 39294), ('time', 35251), ('material', 34208), ('merger', 33843), ('subsidiaries', 33317), ('applicable', 31369), ('including', 29398), ('respect', 28848), ('may', 28069), ('stock', 26651), ('information', 25681), ('parties', 24610), ('b', 23935), ('business', 23497)]
20 Least Common Words: [('122aib', 1), ('122di', 1), ('122ai', 1), ('122aiia', 1), ('122e', 1), ('123ciia2', 1), ('123ciia3', 1), ('123ciib1', 1), ('123ciib2', 1), ('123ciib3', 1), ('123ciii', 1), ('trialbytrial', 1), ('perdetail', 1), ('123cvide', 1), ('123cvi', 1), ('122aiii', 1), ('123di', 1), ('122aiib', 1), ('takings', 1), ('exhibitsancillary', 1)]


#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [30]:
# Transform the page contents of documents

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

contents = [doc.page_content for doc in docs]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(contents)

# Compute similarity scores

# First 10 docs
similarity_matrix_first10 = cosine_similarity(tfidf_matrix[:10])
print(similarity_matrix_first10)


[[1.         0.29298252 0.30119917 0.19987952 0.27135081 0.19520498
  0.17481365 0.10513746 0.05580197 0.06503493]
 [0.29298252 1.         0.40471471 0.32220309 0.42714379 0.28313062
  0.31475707 0.13116893 0.05755453 0.09096209]
 [0.30119917 0.40471471 1.         0.35764548 0.52624268 0.31982757
  0.24753857 0.12046101 0.06402482 0.07828118]
 [0.19987952 0.32220309 0.35764548 1.         0.34305073 0.20130963
  0.19943237 0.06875807 0.03090197 0.04641382]
 [0.27135081 0.42714379 0.52624268 0.34305073 1.         0.30138468
  0.26771973 0.10962853 0.05489759 0.07350297]
 [0.19520498 0.28313062 0.31982757 0.20130963 0.30138468 1.
  0.17732837 0.08294927 0.03999159 0.05439024]
 [0.17481365 0.31475707 0.24753857 0.19943237 0.26771973 0.17732837
  1.         0.06451441 0.03113781 0.04363283]
 [0.10513746 0.13116893 0.12046101 0.06875807 0.10962853 0.08294927
  0.06451441 1.         0.11276923 0.11385125]
 [0.05580197 0.05755453 0.06402482 0.03090197 0.05489759 0.03999159
  0.03113781 0.11276

In [31]:
# create a list of 10 random integers
random_indices = random.sample(range(len(contents)), 10)


In [32]:
# Compute similarity scores for 10 random documents
similarity_matrix_random10 = cosine_similarity(tfidf_matrix[random_indices])
print(similarity_matrix_random10)

[[1.         0.0757301  0.13947774 0.1187125  0.32801922 0.16913765
  0.06886358 0.06086552 0.24267132 0.12255768]
 [0.0757301  1.         0.10467891 0.09211151 0.10088128 0.03822613
  0.0532536  0.05155355 0.13454115 0.09852918]
 [0.13947774 0.10467891 1.         0.16961128 0.20049042 0.07492238
  0.10470828 0.08798864 0.22253915 0.42256896]
 [0.1187125  0.09211151 0.16961128 1.         0.37648659 0.09629327
  0.1010073  0.10773696 0.30676399 0.16787597]
 [0.32801922 0.10088128 0.20049042 0.37648659 1.         0.30297018
  0.10307882 0.1087616  0.45611776 0.19638037]
 [0.16913765 0.03822613 0.07492238 0.09629327 0.30297018 1.
  0.03233631 0.03169758 0.1704716  0.07605157]
 [0.06886358 0.0532536  0.10470828 0.1010073  0.10307882 0.03233631
  1.         0.0566992  0.13616306 0.11665981]
 [0.06086552 0.05155355 0.08798864 0.10773696 0.1087616  0.03169758
  0.0566992  1.         0.13577851 0.11283305]
 [0.24267132 0.13454115 0.22253915 0.30676399 0.45611776 0.1704716
  0.13616306 0.135778

### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [33]:
# Process files and generate chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [34]:
# Fetch your OPENAI API Key as an environment variable
!pip install -q langchain faiss-cpu transformers datasets evaluate rouge-score nltk
!pip install sentence-transformers
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import json
import random
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [35]:
# Initialise an embedding function
!pip uninstall -y sentence-transformers huggingface-hub
!pip install sentence-transformers==2.6.1 huggingface-hub==0.23.1

from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
docs_to_embed = chunks[:1000]  # Adjust if needed

Found existing installation: sentence-transformers 2.6.1
Uninstalling sentence-transformers-2.6.1:
  Successfully uninstalled sentence-transformers-2.6.1
Found existing installation: huggingface-hub 0.23.1
Uninstalling huggingface-hub-0.23.1:
  Successfully uninstalled huggingface-hub-0.23.1
  Using cached sentence_transformers-2.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.23.1-py3-none-any.whl.metadata (12 kB)
Using cached sentence_transformers-2.6.1-py3-none-any.whl (163 kB)
Using cached huggingface_hub-0.23.1-py3-none-any.whl (401 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
diffusers 0.34.0 requires huggingface-hub>=0.27.0, but you have huggingface-hub 0.23.1 which is incompatible.
gradio 5.31.0 requires huggingface-hub>=0.28.1, but you have huggingface-hub 0.23.1 which is incompatible.
peft 0.15.2 requires huggingface_hub>

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [36]:
# Add Chunks to vector DB
from langchain.vectorstores import FAISS

vector_db = FAISS.from_documents(docs_to_embed, embedding_function)
vector_db.save_local("/content/vector_db")

### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [37]:
# Create a RAG chain
!pip uninstall -y transformers
!pip install transformers==4.40.1
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

qa_model = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=qa_model)

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
    return_source_documents=True
)




Found existing installation: transformers 4.40.1
Uninstalling transformers-4.40.1:
  Successfully uninstalled transformers-4.40.1
  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
Using cached transformers-4.40.1-py3-none-any.whl (9.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.15.2 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.23.1 which is incompatible.


In [38]:
query = "Does the NDA grant rights to confidential info?"
response = qa_chain.invoke({"query": query})

print("Answer:", response['result'])
print("Source Docs:", [doc.metadata['file_path'] for doc in response['source_documents']])


Answer: Yes


KeyError: 'file_path'

#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [39]:
# Create a function for question answering
!pip install evaluate
import evaluate

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

In [ ]:
# Example question
# question ="Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"


## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [40]:
# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set
# Example structure
predictions = [
    "The NDA restricts the use of confidential information.",
    "The contract allows termination under specific conditions."
]

references = [
    "The NDA limits how confidential information may be used.",
    "The agreement can be terminated based on outlined conditions."
]

#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [41]:
# Function to evaluate the RAG pipeline
rouge_results = rouge.compute(predictions=predictions, references=references)
print("🔍 ROUGE Evaluation:")
print(rouge_results)

# BLEU expects:
# - `predictions`: list of strings (or token lists)
# - `references`: list of list of references (each reference = list of strings or tokens)

# FIX:
tokenized_preds = predictions  # Just strings, or use .split() below
tokenized_refs = [[ref] for ref in references]  # Wrap each ref in a list

bleu_results = bleu.compute(predictions=tokenized_preds, references=tokenized_refs)
print("🔍 BLEU Evaluation:")
print(bleu_results)


🔍 ROUGE Evaluation:
{'rouge1': np.float64(0.3602941176470588), 'rouge2': np.float64(0.13333333333333333), 'rougeL': np.float64(0.3602941176470588), 'rougeLsum': np.float64(0.3602941176470588)}
🔍 BLEU Evaluation:
{'bleu': 0.0, 'precisions': [0.47058823529411764, 0.2, 0.0, 0.0], 'brevity_penalty': 0.8382234324229999, 'length_ratio': 0.85, 'translation_length': 17, 'reference_length': 20}


#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [42]:
# Evaluate the RAG pipeline

{
  'bleu': 0.0,
  'precisions': [0.47, 0.20, 0.0, 0.0],
  'brevity_penalty': 0.838,
  'length_ratio': 0.85,
  'translation_length': 17,
  'reference_length': 20
}


{'bleu': 0.0,
 'precisions': [0.47, 0.2, 0.0, 0.0],
 'brevity_penalty': 0.838,
 'length_ratio': 0.85,
 'translation_length': 17,
 'reference_length': 20}

## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.